In [1]:
import numpy as np
import matplotlib.pyplot as plt

r0 = 2.818e-13 # classical electron radius in cm
m_e = 9.10938e-28 # electron mass in gram
c = 2.998e10 # speed of light in cm/s
h = 6.6261e-27 # planck constant in cm^2 g s-1
eV_to_erg = 1.6022e-12
parsec_to_cm = 3.085677581e18 # cm per parsec

kB = 1.3808e-16 # boltzmann constant in cm^2 g s^-2 K^-1
T_CMB = 2.725 # CMB temperature today in K

H_0 = 2.184e-18 # current hubble constant in s^-1
H_r = 3.24076e-18 # 100 km/s/Mpc to 1/s
h_0 = H_0 / H_r
omega_b_0 = 0.0224 / h_0 / h_0  # current baryon abundance
m_p = 1.67262193269e-24 # proton mass in g
G = 6.6743e-8 # gravitational constant in cm^3 g^-1 s^-2
f_He = 0.079 # helium mass fraction
f_H = 0.76 # hydrogen fraction
Y_He = 0.24

E_e = m_e * c * c
rho_crit = 3 * H_0 * H_0 / 8 / np.pi / G
e = 4.80326e-10 # electron charge in esu

z = 2
z2 = (1 + z) / 3
T_CMB *= 1 + z 

T_IGM = 9245.6 # IGM temperature in K
# rec coefficients 1991A&A...251..680P
alpha_H = 5.596e-13*(T_IGM/1e4)**(-.6038)/(1. + 0.3436*(T_IGM/1e4)**0.4479)
alpha_He = 5.596e-13*2*(T_IGM/4e4)**(-.6038)/(1. + 0.3436*(T_IGM/4e4)**0.4479)

In [2]:
import IGM

IGM_00 = IGM.IGM_N(0,0)
N_00, P_00, P_00_eV = IGM_00.get_P()
dNdE_z2 = N_00[:,240]
E_z2 = IGM.E[:np.size(dNdE_z2)]
p_grid = np.sqrt(E_z2*(2*m_e+E_z2/c**2))
fp_grid = dNdE_z2/(4.*np.pi*p_grid*(m_e+E_z2/c**2))
slope_fp_grid = np.log(fp_grid[1:]/fp_grid[:-1])/np.log(p_grid[1:]/p_grid[:-1])

#print(p_grid[::10], 4.*np.pi*p_grid[::10]**3*fp_grid[::10], fp_grid[::10], slope_fp_grid[::10])
print(IGM_00.z[240])

2.0083681725319145


In [3]:
# upsilon is a regularization here.
# should converge as upsilon -> 0;
#  a good choice is the step size in the integral
def D_R(y, upsilon = .01):
    upsilon_scale = upsilon * (2*y/(1+y**2))**2
    return 1 + .5 / y * np.log(np.sqrt((1 - y)**2 + upsilon_scale**2) / (1 + y))


In [4]:
# vectorized version so we can give an array to y
# (the if-else would have to be done element by element)
def D_I(y):
    return -np.pi/2/y*np.heaviside(y-1,.5)
    #if y-1 <= 0:
    #    return 0
    #else:
    #    return -np.pi / 2 / y

In [5]:
### as we only focus on z = 2, where is fully ionized, I only consider electrons
### do we want to implement general formula for others to use if needed?
### we want the ions (H+ and He2+) as well

omega_b = omega_b_0 * (1 + z)**3
mean_n_b = 3 * omega_b * H_0 * H_0 / (8 * np.pi * G * m_p) # mean number density of baryons at the redshift
n_b = mean_n_b # for mean density
    
n_H = n_b * (1 - Y_He) # nuclei in hydrogen
n_He = n_H * f_He # nuclei in helium, assume He is all 4He

n_e = n_H + 2 * n_He # for fully ionized

In [6]:
# photoionization-induced electrons

# rates in cm^-3 s^-1
rate_ion_H = alpha_H * n_e * n_H
rate_ion_He = alpha_He * n_e * n_He
#print(rate_ion_H, rate_ion_He)

Ry = m_e*c**2/137.06**2/2. #Rydberg energy

# get spectra from file
bgspec = np.loadtxt('KS_2018_EBL/Fiducial_Q18/EBL_KS18_Q18_z_ 2.0.txt')[::-1,:]
bgspec_E = h*c/(bgspec[:,0]*1e-8)
bgspec_J = bgspec[:,1]
# return photoionization cross section, un-normalized, set to constant below threshold so it can be interpolated
def sigpi(Z):
    npr2inv = bgspec_E/(Z**2*Ry)-1.
    npr = npr2inv**(-.5)
    X2 = np.where(npr2inv>0, np.exp(4.-4*npr*np.arctan(1/npr))/(1.-np.exp(-2*np.pi*npr))/(1.+npr2inv)**5,1.)
    return X2
sigma_pi = [0, sigpi(1), sigpi(2)]

# rate of photoionization producing electrons at momentum > p_e
# (in cm^-3 s^-1)
#def rate_gt_E1(p_e):
#    # loop over hydrogen and helium
#    for Z in [1,2]:
#        eph = p_e**2/2/m_e + Z**2*Ry
#        deph = np.zeros_like(eph)
#        deph[:-1] = eph[1:]-eph[:-1]
#        deph[-1] = eph[-1]-eph[-2]
#        this_sigma = np.interp(eph+.5*deph,bgspec_E,sigma_pi[Z],right=0.)
#        this_rate = this_sigma*deph/eph # contribution to the rate between eph[i] and eph[i+1]
#        this_rate /= np.sum(this_rate)
#        if Z==1: tot_rate = rate_ion_H*this_rate
#        if Z==2: tot_rate = tot_rate + rate_ion_He*this_rate
#    # now do the cumulative counts in reverse
#    return np.cumsum(tot_rate[::-1])[::-1]
def rate_gt_E(p_e):
    # loop over hydrogen and helium
    tot_rate = np.zeros(np.size(p_e)+1)
    p = np.zeros(np.size(p_e)+1)
    p[:-1] = p_e # append 0 so we have the total rate
    Np = 100
    factor = 1.e2 # Emax/Emin
    e__emin = factor**np.linspace(.5/Np,1-.5/Np,Np)
    for Z in [1,2]:
        eph0 = p**2/2/m_e + Z**2*Ry
        for ie in range(Np):
            eph = eph0*e__emin[ie]
            deph = eph*np.log(factor)/Np
            this_sigma = np.interp(eph,bgspec_E,sigma_pi[Z],right=0.)
            this_rate = this_sigma*deph/eph # contribution to the rate between eph[i] and eph[i+1]
        this_rate /= this_rate[-1]
        if Z==1: tot_rate = tot_rate + rate_ion_H*this_rate
        if Z==2: tot_rate = tot_rate + rate_ion_He*this_rate
    return tot_rate[:-1]

# now suppose we have J_bk ~ nu^{-alpha_b}
#alpha_b = 1.
# rate of absorption at energy >E_e is int nu^{-3-alpha_b} dnu/nu ~ nu^{-3-alpha_b}
#def rate_gt_E(p_e):
#    E_e = p_e**2/(2*m_e)
#    return (rate_ion_H*(1.+E_e/Ry)**(-3.-alpha_b) + rate_ion_He*(1.+E_e/4./Ry)**(-3.-alpha_b))


# dynamical friction
def loss_dpdt(p_e):
    # formula: -dp/dt = 4 pi n_e e^4 m_e / p_e^2 * ln Lambda
    # Lambda = rmax/rmin = 1/(kD*rmin)
    kD = np.sqrt(4.*np.pi*e**2/kB/T_IGM*(n_e+n_H+4*n_He))
    rmin = e**2/(p_e**2/(2*m_e))
    return 4*np.pi*n_e*e**4*m_e/p_e**2*np.log(1/kD/rmin)

# phase space density
def fp_photoionization(p_e):
    return rate_gt_E(p_e)/(4.*np.pi*p_e**2*loss_dpdt(p_e))

p_e__ = m_e*c*np.logspace(-4,4,81)
rateg = rate_gt_E(p_e__)
pdot = loss_dpdt(p_e__)
fpi = fp_photoionization(p_e__)
print('# ', rate_ion_H, rate_ion_He)
for i in range(81):
    print('{:12.5E} {:12.5E} {:12.5E} {:12.5E}'.format(p_e__[i],p_e__[i]**2/(2*m_e)/eV_to_erg,rateg[i],fpi[i]))


#  1.3593453019917912e-23 5.606301941345323e-24
 2.73099E-21  2.55509E-03  1.91870E-23  1.91291E+43
 3.43812E-21  4.04954E-03  1.91795E-23  1.87281E+43
 4.32833E-21  6.41808E-03  1.91677E-23  1.83391E+43
 5.44905E-21  1.01720E-02  1.91490E-23  1.79590E+43
 6.85994E-21  1.61215E-02  1.91193E-23  1.75836E+43
 8.63616E-21  2.55509E-02  1.90722E-23  1.72068E+43
 1.08723E-20  4.04954E-02  1.89976E-23  1.68196E+43
 1.36874E-20  6.41808E-02  1.88793E-23  1.64087E+43
 1.72314E-20  1.01720E-01  1.86919E-23  1.59536E+43
 2.16930E-20  1.61215E-01  1.83948E-23  1.54226E+43
 2.73099E-20  2.55509E-01  1.79240E-23  1.47669E+43
 3.43812E-20  4.04954E-01  1.71778E-23  1.39106E+43
 4.32833E-20  6.41808E-01  1.60111E-23  1.27482E+43
 5.44905E-20  1.01720E+00  1.49158E-23  1.16801E+43
 6.85994E-20  1.61215E+00  1.31798E-23  1.01531E+43
 8.63616E-20  2.55509E+00  1.06983E-23  8.10979E+42
 1.08723E-19  4.04954E+00  8.34361E-24  6.22539E+42
 1.36874E-19  6.41808E+00  5.88462E-24  4.32269E+42
 1.72314E-19  1.

/var/folders/g5/l54wg2r14cvdshltwdph6yf80000gn/T/ipykernel_83467/2155501712.py:17: RuntimeWarning: invalid value encountered in power
  npr = npr2inv**(-.5)


In [7]:
# species index i = 0 (e), 1 (p), 2 (alpha)
nsp = 3 # number of species
mass_sp = np.array([m_e, m_p, 4*m_p]) # masses
nth_sp = np.array([n_e,n_H,n_He]) # number densities
Z_sp = np.array([-1.,1.,2.]) # charges

# gamma determined from momentum p (g cm/s) and index i
def gamma_from_p(p,i):
    return(np.sqrt(1. + (p/(c*mass_sp[i]))**2))

def v_from_p(p,i):
    return(p/mass_sp[i]/np.sqrt(1. + (p/(c*mass_sp[i]))**2))

# phase space density
def f0_from_p(p,i,return_all=False):
    x = p/np.sqrt(2*kB*T_IGM*mass_sp[i])
    fth = nth_sp[i]*(2*np.pi*kB*T_IGM*mass_sp[i])**(-1.5)*np.exp(-x*x)
    ftot = np.copy(fth)
    if i==0:
        nExt = 60
        p_grid2 = np.zeros((np.size(p_grid)+nExt))
        fp_grid2 = np.zeros((np.size(p_grid)+nExt))
        p_grid2[nExt:] = p_grid
        fp_grid2[nExt:] = fp_grid
        p_th = np.sqrt(2*kB*T_IGM*m_e)
        if p_th>=.5*p_grid[0]: p_th=.5*p_grid[0]
        p_grid2[:nExt] = p_th * (p_grid[0]/p_th)**np.linspace(0,1,nExt+1)[:-1]
        fp_grid2[:nExt] = fp_grid[0]*(p_grid2[:nExt]/p_grid[0])**(-2.)
        fcr = np.interp(p,p_grid2,fp_grid2,right=0.)
        fpi = fp_photoionization(p)
        ftot = fth + fcr + fpi
    if return_all and i==0: return ftot,fth,fcr,fpi
    return(ftot)

def df0dp_from_p(p,i,dpp=.01):
    return((f0_from_p(p*(1+dpp),i)-f0_from_p(p*(1-dpp),i))/(2*p*dpp))

# this is to generate some output data files for the electron distribution,
# not needed for normal operation
p = np.logspace(-22,-12,1001) # this was for test only
f1tot, f1th, f1cr, f1pi = f0_from_p(p,0,return_all=True)
#for i in range(801):
#     print('{:12.5E} {:12.5E} {:12.5E} {:12.5E} {:12.5E} {:12.5E} {:12.5E}'.format(p[i], (gamma_from_p(p,0)[i]-1)*m_e*c**2/eV_to_erg,
#        f0_from_p(p,0)[i], f1th[i], f1cr[i], f1pi[i], df0dp_from_p(p,0)[i]))
np.savetxt('phasespace_e_z2.dat', np.stack((p, (gamma_from_p(p,0)-1)*m_e*c**2/eV_to_erg,
    f0_from_p(p,0), f1th, f1cr, f1pi, df0dp_from_p(p,0))).T)

# given v; I don't use p as it's too small to be calculated reasonablely
# OK as long as multiplications are done carefully to avoid overflow/underflow

def gamma(v_p):
    if abs(v_p**2 / c**2 - 1) <= 1e-28:
        return 1e14
    if v_p > c:
        print(v_p)
        return 1e14
    return 1 / np.sqrt(1 - v_p**2 / c**2)

def f0(v_p): # is the T there T_CMB or T_IGM?
    return np.sqrt(m_e / (2 * np.pi * kB * T_CMB)) * np.exp(-m_e * v_p**2 / (2 * kB * T_CMB))

def df0(v_p):
    return -2 * m_e * v_p / (2 * kB * T_CMB) * f0(v_p)

In [8]:
v_p_max = c

def G_a_cons(v_ph, isp=0): # conservative response
    #v_p = np.logspace(0, np.log10(v_p_max), 400)
    p = c * mass_sp[isp] * np.logspace(-11,7,18001)
    dlnp = np.log(p[1]/p[0])
    v_p = v_from_p(p,isp)
    #int_p = 0
    #for i in range(1, 400):
    #    int_i = D_R(v_p[i] / v_ph) * p[i] * gamma(v_p[i]) * df0(v_p[i]) * (p[i] - p[i-1])
    #    int_p += int_i
    # ** alternative method using vectorized arithmetic **
    int_p = dlnp*np.sum( D_R(v_p/v_ph) * p * gamma_from_p(p,isp) * df0dp_from_p(p,isp) * p )
    # *p at the end is for integration measure
    return 4 * np.pi * mass_sp[isp]**2 * v_ph**2 / nth_sp[isp] * int_p

def g_a_dist(v_ph, isp=0): # projected distribution
    # exit if phase velocity is too large, nothing resonant
    if 1.-v_ph**2 / c**2 <= 1e-14:
        return 0.
    gamma_ph = 1 / np.sqrt(1 - v_ph**2 / c**2)
    p_min = mass_sp[isp] * v_ph * gamma_ph
    v_p_min = p_min / m_e
    p = p_min * np.logspace(0.0005,9.9995,10000)
    dlnp = np.log(p[1]/p[0])
    #if v_p_min > c:
    #    return 0
    #v_p = np.logspace(np.log10(v_p_min), np.log10(v_p_max), 400)
    #p = v_p * m_e
    #int_p = 0
    #for i in range(1, 400):
    #    int_i = f0(v_p[i]) * p[i] * gamma(v_p[i]) * (p[i] - p[i-1])
    #    int_p += int_i
    # ** alternative method using vectorized arithmetic **
    int_p = dlnp*np.sum( f0_from_p(p,isp) * p * gamma_from_p(p,isp) * p )
    # *p at the end is for integration measure
    return 4 * np.pi * mass_sp[isp] / nth_sp[isp] * int_p

def g_a_damp(v_ph, isp=0): # damping response
    # exit if phase velocity is too large, nothing resonant
    if 1.-v_ph**2 / c**2 <= 1e-14:
        return 0.
    gamma_ph = 1 / np.sqrt(1 - v_ph**2 / c**2)
    p_min = mass_sp[isp] * v_ph * gamma_ph
    v_p_min = p_min / m_e
    p = p_min * np.logspace(0.0005,9.9995,10000)
    dlnp = np.log(p[1]/p[0])
    #if v_p_min > c:
    #    return 0
    #v_p = np.logspace(np.log10(v_p_min), np.log10(v_p_max), 400)
    #p = v_p * m_e
    #int_p = 0
    #for i in range(1, 400):
    #    int_i = df0(v_p[i]) / v_p[i] * p[i] * gamma(v_p[i]) * (p[i] - p[i-1])
    #    int_p += int_i
    # ** alternative method using vectorized arithmetic **
    int_p = dlnp*np.sum( df0dp_from_p(p,isp)/v_from_p(p,isp) * p * gamma_from_p(p,isp) * p )
    # *p at the end is for integration measure
    return -4 * np.pi * mass_sp[isp] / nth_sp[isp] * int_p

#for j in range(81):
#    v_ph = 10.**(3+.1*j)
#    print('{:12.5E} {:12.5E} {:12.5E} {:12.5E}'.format(v_ph, G_a_cons(v_ph), g_a_dist(v_ph), g_a_damp(v_ph)))

In [9]:
### does e the nature log base?
Z_e = 1
def G_cons(v_ph):
    tot = 0.
    for isp in range(nsp):
        tot += 4 * np.pi * e**2 * Z_sp[isp]**2 * nth_sp[isp] / mass_sp[isp] * G_a_cons(v_ph,isp)
    return tot

def g_dist(v_ph):
    tot = 0.
    den = 0.
    for isp in range(nsp):
        tot += Z_sp[isp]**2 * nth_sp[isp] * g_a_dist(v_ph,isp)
        den += Z_sp[isp]**2 * nth_sp[isp]
    return(tot/den)

def g_damp(v_ph):
    tot = 0.
    for isp in range(nsp):
        tot += 2 * np.pi**2 * e**2 * v_ph * Z_sp[isp]**2 * nth_sp[isp] * g_a_damp(v_ph,isp)
    return(tot)

for j in range(81):
    v_ph = 10.**(3+.1*j)
    print('{:12.5E} {:12.5E} {:12.5E} {:12.5E}'.format(v_ph, G_cons(v_ph), g_dist(v_ph), g_damp(v_ph)))

# output file
# columns: v_ph; Gcons e,p,alpha; Gcons; gdist e,p,alpha; gdist; gdamp e,p,alpha; gdamp
allgfunc = np.zeros((801,13))
for j in range(801):
    v_ph = 10.**(3+.01*j)
    allgfunc[j,0] = v_ph
    for isp in range(3):
        allgfunc[j,1+isp] = 4 * np.pi * e**2 * Z_sp[isp]**2 * nth_sp[isp] / mass_sp[isp] * G_a_cons(v_ph,isp)
    allgfunc[j,4] = G_cons(v_ph)
    for isp in range(3):
        allgfunc[j,5+isp] = Z_sp[isp]**2 * nth_sp[isp] * g_a_dist(v_ph,isp) / np.sum(Z_sp**2*nth_sp)
    allgfunc[j,8] = g_dist(v_ph)
    for isp in range(3):
        allgfunc[j,9+isp] = 2 * np.pi**2 * e**2 * v_ph * Z_sp[isp]**2 * nth_sp[isp] * g_a_damp(v_ph,isp)
    allgfunc[j,12] = g_damp(v_ph)
np.savetxt('allgfunc.dat', allgfunc)

 1.00000E+03 -2.90039E-05  6.12433E-07  2.79001E-14
 1.25893E+03 -4.59680E-05  6.12432E-07  3.51241E-14
 1.58489E+03 -7.28543E-05  6.12432E-07  4.42186E-14
 1.99526E+03 -1.15466E-04  6.12430E-07  5.56678E-14
 2.51189E+03 -1.83001E-04  6.12428E-07  7.00814E-14
 3.16228E+03 -2.90036E-04  6.12425E-07  8.82267E-14
 3.98107E+03 -4.59673E-04  6.12420E-07  1.11070E-13
 5.01187E+03 -7.28524E-04  6.12412E-07  1.39827E-13
 6.30957E+03 -1.15461E-03  6.12400E-07  1.76028E-13
 7.94328E+03 -1.82989E-03  6.12380E-07  2.21599E-13
 1.00000E+04 -2.90004E-03  6.12348E-07  2.78962E-13
 1.25893E+04 -4.59594E-03  6.12298E-07  3.51164E-13
 1.58489E+04 -7.28326E-03  6.12219E-07  4.42033E-13
 1.99526E+04 -1.15412E-02  6.12094E-07  5.56372E-13
 2.51189E+04 -1.82864E-02  6.11896E-07  7.00204E-13
 3.16228E+04 -2.89692E-02  6.11581E-07  8.81052E-13
 3.98107E+04 -4.58810E-02  6.11084E-07  1.10828E-12
 5.01187E+04 -7.26360E-02  6.10296E-07  1.39344E-12
 6.30957E+04 -1.14919E-01  6.09051E-07  1.75066E-12
 7.94328E+04

In [10]:
def F_NP(k_perp):
    v_proj = c # for now
    Nv = 4001
    v_ph = np.logspace(0, np.log10(v_proj), Nv)
    
    # we want to do the integral in a way that properly samples the resonance at low k
    # if we integrate a function of the form int_{v_i}^{v_{i+1}} C1/(C2^2+C3^2) dv
    # where the C's vary slowly but C2 may pass through zero, we write C1,C3 = const, C2 = linear fcn of v
    # then this leads to (for C3>0)
    # C1 / C3 * ( tan^-1(C2/C3)[i+1] - tan^-1(C2/C3)[i] ) / (dC2/dv)
    # but this is numerically unstable if C3 is very small
    int_v_ph = 0
    C1 = np.zeros((Nv,))
    C2 = np.zeros((Nv,))
    C3 = np.zeros((Nv,))
    for i in range(Nv):
        co_ph = 1 - v_ph[i]**2 / v_proj**2
        C1[i] = co_ph * g_dist(v_ph[i])
        C2[i] = 1 - k_perp**(-2) * v_ph[i]**(-2) * co_ph * G_cons(v_ph[i])
        C3[i] = k_perp**(-2) * co_ph * g_damp(v_ph[i])
    C3 = C3 + 1e-200 # <-- forces a floor to the damping to get rid of division by zero
        # (shouldn't be necessary when we include CRs)
    int_v_ph = 0.
    for i in range(Nv-1):
        is_nonres = True
        if C2[i]*C2[i+1]<0: is_nonres=False
        if i<Nv-2:
            if C2[i+1]*C2[i+2]<0: is_nonres=False
        if i>0:
            if C2[i-1]*C2[i]<0: is_nonres=False
        if is_nonres:
            int_v_ph += (C1[i]+C1[i+1])/2/( ((C2[i]+C2[i+1])/2)**2+((C3[i]+C3[i+1])/2)**2 )*(v_ph[i+1]-v_ph[i])
        else:
            #print('res', i, v_ph[i:i+2], C1[i:i+2], C2[i:i+2], C3[i:i+2])
            int_v_ph += (C1[i]+C1[i+1])/(C3[i]+C3[i+1]) * (np.arctan2(C2[i+1],C3[i+1])-np.arctan2(C2[i],C3[i]))/\
                (C2[i+1]-C2[i])*(v_ph[i+1]-v_ph[i])
        # co1 / (co2 + co3) * (v_ph[i+1] - v_ph[i-1])/2. # latter is effective dv_ph
    return int_v_ph

In [11]:
k_perp = np.logspace(-9, -3, 301)
k_D = np.sqrt(4.*np.pi*e**2/kB/T_IGM*np.sum(nth_sp*Z_sp**2))
print(k_D)

5.385257015560704e-06


In [12]:
F_NP_K = np.zeros((np.size(k_perp), ))
for i in range(np.size(k_perp)):
    F_NP_K[i] = F_NP(k_perp[i])
    #print(f'i = {i}, F_NP_Ki = {F_NP_K[i]}')
    print('{:12.5E} {:12.5E}'.format(k_perp[i],F_NP_K[i]))
np.savetxt('fnpk_z2.dat', np.stack((k_perp,F_NP_K)).T)

 1.00000E-09  1.24280E-03
 1.04713E-09  1.31142E-03
 1.09648E-09  1.57667E-03
 1.14815E-09  1.69759E-03
 1.20226E-09  2.04096E-03
 1.25893E-09  2.45377E-03
 1.31826E-09  2.68075E-03
 1.38038E-09  2.95900E-03
 1.44544E-09  3.55750E-03
 1.51356E-09  3.95589E-03
 1.58489E-09  4.42688E-03
 1.65959E-09  4.98273E-03
 1.73780E-09  5.63245E-03
 1.81970E-09  6.38454E-03
 1.90546E-09  7.26091E-03
 1.99526E-09  8.28437E-03
 2.08930E-09  9.47216E-03
 2.18776E-09  1.08494E-02
 2.29087E-09  1.18962E-02
 2.39883E-09  1.36870E-02
 2.51189E-09  1.51209E-02
 2.63027E-09  1.67951E-02
 2.75423E-09  1.87074E-02
 2.88403E-09  2.09073E-02
 3.01995E-09  2.34188E-02
 3.16228E-09  2.54380E-02
 3.31131E-09  2.86525E-02
 3.46737E-09  3.13353E-02
 3.63078E-09  3.43832E-02
 3.80189E-09  3.78371E-02
 3.98107E-09  4.17429E-02
 4.16869E-09  4.61553E-02
 4.36516E-09  4.97822E-02
 4.57088E-09  5.52849E-02
 4.78630E-09  5.99412E-02
 5.01187E-09  6.51539E-02
 5.24807E-09  6.92607E-02
 5.49541E-09  7.56263E-02
 5.75440E-09

In [13]:
dlnk_perp = np.log(k_perp[1]/k_perp[0])
kmin_eff = k_perp[-1]*np.exp(dlnk_perp*(.5-np.sum(F_NP_K)))
print(kmin_eff, np.log(k_D/kmin_eff))

2.420555386408285e-07 3.102253118161918
